In [64]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional,Dense,LSTM,Embedding,Dropout
from sklearn.model_selection import train_test_split

## Cleaned my DataSet

In [41]:
df=pd.read_csv('movie_review.csv')
df.columns

Index(['fold_id', 'cv_tag', 'html_id', 'sent_id', 'text', 'tag'], dtype='object')

In [42]:
df.drop(columns=['fold_id', 'cv_tag', 'html_id', 'sent_id'],inplace=True)

In [43]:
df['tag'] = df['tag'].map({'pos': 0, 'neg': 1})


In [44]:
df

,text,tag
0,films adapted from comic books have had plenty...,0
1,"for starters , it was created by alan moore ( ...",0
2,to say moore and campbell thoroughly researche...,0
3,"the book ( or "" graphic novel , "" if you will ...",0
4,"in other words , don't dismiss this film becau...",0
...,...,...
64715,that lack of inspiration can be traced back to...,1
64716,like too many of the skits on the current inca...,1
64717,"after watching one of the "" roxbury "" skits on...",1
64718,"bump unsuspecting women , and . . . that's all .",1


## Text Vectorization

In [45]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [46]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['text'])

In [47]:
total_words=len(tokenizer.word_index)+1


In [49]:
seq=tokenizer.texts_to_sequences(df['text'])
padded=pad_sequences(seq,padding='pre')


In [ ]:
max_seq_len=padded.shape[1]

## Model Training

In [54]:
X_train, X_test, y_train, y_test = train_test_split(padded, df['tag'], test_size=0.2, random_state=42)

In [ ]:
model =Sequential()
model.add(Embedding(input_dim=total_words,output_dim=100,input_length=max_seq_len-1))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(1,activation='sigmoid'))

c:\Users\jatin\OneDrive\Desktop\RNN\sentiment_analysis\venv\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [66]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.build(input_shape=(None,max_seq_len-1))

In [67]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 151, 100)       │     4,329,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 151, 256)       │       234,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 151, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 256)            │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,958,693 (18.92 MB)

 Trainable params: 4,958,693 (18.92 MB)

 Non-trainable params: 0 (0.00 B)

In [69]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',         
    patience=5,            
    restore_best_weights=True
)